In [2]:
import tkinter as tk
from PIL import Image, ImageTk, ImageChops

In [37]:
class TKcanvas_setup(tk.Toplevel):
    def __init__(self, setup_image, master):
#         tk.Toplevel.__init__(self)
        #tk.Tk.__init__(self)
        self.x = self.y = 0
        self.image = ImageTk.PhotoImage(setup_image, master) # Use correct master else create_image fails (when Toplevel not used)
        self.canvas = tk.Canvas(master, width=self.image.width(), height=self.image.height(), cursor="cross")
        self.canvas.pack(side="top", fill="both", expand=True)
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)
        self.canvas.bind("<B1-Motion>", self.on_button_move)
        self.canvas.create_image((self.image.width() / 2, self.image.height() / 2), image=self.image)
        self.lines = []
        # Can combine below variables into struct & create a list of struct instead of "list of each var"
        self.obj_list = []
        self.full_obj_marker_list = []
        self.saved_rect_list = []
        self.saved_obj_text = []

    def on_button_press(self, event):
        self.x = event.x
        self.y = event.y

    def on_button_move(self, event):
        x0, y0 = (self.x, self.y)
        x1, y1 = (event.x, event.y)

        # Remove lines
        for line_id in self.lines:
            self.canvas.delete(line_id)

        self.lines = []

        self.lines.append(self.canvas.create_line(x0, y0, x0, y1, fill="black"))
        self.lines.append(self.canvas.create_line(x0, y1, x1, y1, fill="black"))
        self.lines.append(self.canvas.create_line(x1, y1, x1, y0, fill="black"))
        self.lines.append(self.canvas.create_line(x1, y0, x0, y0, fill="black"))

    def on_button_release(self, event):
        x0, y0 = (self.x, self.y)
        x1, y1 = (event.x, event.y)

        for line_id in self.lines:
            self.canvas.delete(line_id)

        self.lines = []

        self.lines.append(self.canvas.create_line(x0, y0, x0, y1, fill="red"))
        self.lines.append(self.canvas.create_line(x0, y1, x1, y1, fill="red"))
        self.lines.append(self.canvas.create_line(x1, y1, x1, y0, fill="red"))
        self.lines.append(self.canvas.create_line(x1, y0, x0, y0, fill="red"))
#       self.canvas.create_rectangle(x0, y0, x1, y1, fill="black")

        self.full_obj_marker = (x0, y0, x1, y1)
        obj_name = self.get_object_info(())
    
    def get_object_info(self, full_obj_marker):
        obj_entry_box = tk.Tk()
        tk.Label(obj_entry_box, text="Item Name").grid(row=0)
        self.e1 = tk.Entry(obj_entry_box)
        self.e1.grid(row=0, column=1)
        tk.Button(obj_entry_box, text='Save', command=self.save_marker).grid(row=2, column=0, sticky=tk.W, pady=4)
        tk.Button(obj_entry_box, text='Quit', command=obj_entry_box.destroy).grid(row=2, column=1, sticky=tk.W, pady=4)
        tk.Button(obj_entry_box, text='Delete', command=self.del_marker).grid(row=2, column=3, sticky=tk.W, pady=4)

    def save_marker(self):
        obj_name = self.e1.get()
        if (obj_name not in self.obj_list and self.full_obj_marker not in self.full_obj_marker_list):
            self.obj_list.append(obj_name)
            self.full_obj_marker_list.append(self.full_obj_marker)
            self.saved_rect_list.append(self.canvas.create_rectangle(self.full_obj_marker, outline="green"))
            center_x = (self.full_obj_marker[0]+self.full_obj_marker[2])/2
            center_y = (self.full_obj_marker[1]+self.full_obj_marker[3])/2
            self.saved_obj_text.append(self.canvas.create_text(center_x, center_y, fill="green",
                                                               font="Times 20 italic bold", text=obj_name))
        else:
            # Add another dialog box that warns user that the obj_name already exists
            print ("Warning: Object name or Marker already present")
    
    def del_marker(self):
        obj_name = self.e1.get()
        if obj_name in self.obj_list:  
            obj_idx  = self.obj_list.index(obj_name)
            self.obj_list.pop(obj_idx)
            self.full_obj_marker_list.pop(obj_idx)
            self.canvas.delete(self.saved_rect_list[obj_idx])
            self.saved_rect_list.pop(obj_idx)
            self.canvas.delete(self.saved_obj_text[obj_idx])
            self.saved_obj_text.pop(obj_idx)
        else:
            # Add another dialog box that warns user that the obj_name already exists
            print ("Warning: Object name not present")

In [42]:
# SETUP BOXES
setup_image = Image.open("test.jpg")
#root = tk.Toplevel()

root = tk.Tk()
TKcanvas_setup_inst = TKcanvas_setup(setup_image, root)
root.mainloop()

In [54]:
# SEND BOX COORDINATES TO DYNAMODB TABLE
print (TKcanvas_setup_inst.obj_list)

['Lot1', 'Lot2', 'Lot3', 'Lot4', 'Lot5', 'Lot6']


In [60]:
import json
boxes = {}
for (idx,region) in enumerate(TKcanvas_setup_inst.obj_list):
    boxes[region] = list(TKcanvas_setup_inst.full_obj_marker_list[idx])
print(json.dumps(boxes))

{"Lot1": [32, 264, 264, 515], "Lot2": [200, 402, 404, 656], "Lot3": [360, 570, 586, 799], "Lot4": [624, 4, 906, 206], "Lot5": [771, 120, 1041, 368], "Lot6": [959, 251, 1200, 493]}


In [1]:
import boto3
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
table = dynamodb.Table('CE_Status_Table')

ModuleNotFoundError: No module named 'boto3'

In [2]:
arr = [{'Name': 'Bus', 'Confidence': 61.101375579833984}, {'Name': 'Vehicle', 'Confidence': 61.101375579833984}, {'Name': 'Moving Van', 'Confidence': 55.52613067626953}, {'Name': 'Moving Van', 'Confidence': 55.52613067626953}, {'Name': 'Van', 'Confidence': 55.52613067626953}]

In [5]:
import datetime
from pytz import timezone
import pytz

date = datetime.datetime.now().astimezone(pytz.timezone('US/Pacific'))
timestamp = date.strftime('%Y-%m-%dT%H:%M:%S')

print ('Local date & time is  :', timestamp)


Local date & time is  : 2018-07-12T18:44:07


In [6]:
utc_date = pytz.timezone('UTC').localize(datetime.datetime.utcnow())
local_date = utc_date.astimezone(pytz.timezone('US/Pacific'))
timestamp = local_date.strftime('%Y-%m-%dT%H:%M:%S')
print ('Local date & time is  :', timestamp)


Local date & time is  : 2018-07-12T18:44:12
